# Image and text Extraction

In [ ]:
from image_processor import extract_image
import cv2

data_path = "../dataset/facticia_3_2/"
image_name = "4c76a3e4-ohcbh_cf_erl8_204"

label_path = f"{data_path}/labels/{image_name}.txt"
image_path = f"{data_path}/images/{image_name}.jpg"

images = []

with open (label_path, "r") as file:
    while True:
        
        line = file.readline()
        
        elements = line.split(" ")
        
        if elements[0] == "2":
            print(elements)
            image = extract_image(image_path, elements[1:])
            images.append(image)
        
        if not line:
            break
        